In [21]:
import numpy as np
import pandas as pd
import os, glob
import nibabel as nib
import math

In [22]:
# Call n subjects
dilated_rec_dir = 'Directory to dilated Rec ROI images'
seg_dir = 'Directory to Segmentation'

segm_dir = sorted(glob.glob(os.path.join(seg_dir, "*_seg*.nii.gz")))
rec_dir = sorted(glob.glob(os.path.join(dilated_rec_dir, "*REC*.nii.gz"))) 

In [23]:
# Load nifti files into numpy array
def LoadingImage(dir):
    
    nifti_image = nib.load(dir)
    image = np.asarray(nifti_image.dataobj)
    header = nifti_image.header
    imgaffine = nifti_image.affine
    
    return image, header, imgaffine

In [24]:
# Get unique subject ID
def getFilename(full_dir):
    _,filename = full_dir.split('\\')
    print(filename)
    subject, _, _, _, _= filename.split('_')
    return str(subject)

In [ ]:
#loop each patients
# far = 2, near = 1, otherwise = 0

outer_dist = 20
uncertain_dist = 7
tumor_dist = 5

# Rotate through each subjects in institution
for i in range(len(segm_dir)):

    # call all the images
    seg_img, seg_hdr, seg_imgaffine = LoadingImage(segm_dir[i])
    #roi
    # call rec image (240, 240, 155)
    rec_img, rec_hdr, rec_imgaffine = LoadingImage(rec_dir[i])

    #create empty array
    empty_array = np.zeros((240, 240, 155))

    #collect coordinates of the edema from seg
    edema_roi_coord=[[x,y,z] for z in range(0,seg_img.shape[2],1) for y in range(0,seg_img.shape[1],1) for x in range(0,seg_img.shape[0],1) if seg_img[x,y,z] == 2]

    #collect coordinates of the tumor from seg
    tumor_roi_coord=[[x,y,z] for z in range(0,seg_img.shape[2],1) for y in range(0,seg_img.shape[1],1) for x in range(0,seg_img.shape[0],1) if seg_img[x,y,z] == 4 or seg_img[x,y,z] == 1]

    # collect coordinates of the rec
    rec_roi_coord=[[x,y,z] for z in range(0,seg_img.shape[2],1) for y in range(0,seg_img.shape[1],1) for x in range(0,seg_img.shape[0],1) if rec_img[x,y,z] > 0.9 and rec_img[x,y,z] < 1.1 and seg_img[x,y,z] == 2]

    # collect coordinates of the brain
    brain_roi_coord=[[x,y,z] for z in range(0,seg_img.shape[2],1) for y in range(0,seg_img.shape[1],1) for x in range(0,seg_img.shape[0],1) if seg_img[x,y,z] != 0 and rec_img[x,y,z] != 1]


    #limited the edema outskirt
    for edema_voxel in edema_roi_coord:
        for tumor_voxel in tumor_roi_coord:
            distance_edema_tumor = math.dist(edema_voxel,tumor_voxel)
            if distance_edema_tumor < outer_dist:
                x = edema_voxel[0]
                y = edema_voxel[1]
                z = edema_voxel[2]
                empty_array[x,y,z] = np.int16(2)

    #case reccurence roi with 1
    for rec_voxel in rec_roi_coord:
        x = rec_voxel[0]
        y = rec_voxel[1]
        z = rec_voxel[2]
        empty_array[x,y,z] = np.int16(1)

    #collect coordinates of the edema from sempty
    edema20_roi_coord=[[x,y,z] for z in range(0,seg_img.shape[2],1) for y in range(0,seg_img.shape[1],1) for x in range(0,seg_img.shape[0],1) if empty_array[x,y,z] > 1.9 and empty_array[x,y,z] < 2.1]

    #0 the space between dilated rec and edema
    for edema_voxel in edema20_roi_coord:
        for rec_voxel in rec_roi_coord:
            distance_edema_rec = math.dist(edema_voxel,rec_voxel)
            if distance_edema_rec < uncertain_dist:
                x = edema_voxel[0]
                y = edema_voxel[1]
                z = edema_voxel[2]
                empty_array[x,y,z] = np.int16(0)

    #collect coordinates of the edema from empty2
    edema_dil_roi_coord=[[x,y,z] for z in range(0,seg_img.shape[2],1) for y in range(0,seg_img.shape[1],1) for x in range(0,seg_img.shape[0],1) if empty_array[x,y,z] > 1.9 and empty_array[x,y,z] < 2.1]

    #0 out pertumoral tissue
    for edema_voxel in edema_dil_roi_coord:
        for tumor_voxel in tumor_roi_coord:
            distance_edema_tumor = math.dist(edema_voxel,tumor_voxel)
            if distance_edema_tumor < tumor_dist:
                x = edema_voxel[0]
                y = edema_voxel[1]
                z = edema_voxel[2]
                empty_array[x,y,z] = np.int16(0)

    # filename
    filename = getFilename(segm_dir[i])

    #save the distance map
    nifti_prob = nib.Nifti1Image(empty_array, seg_imgaffine, header = seg_hdr)
    dir_path = 'Save it into desired ouput directory for evaluation maps'
    nib.save(nifti_prob, dir_path + 'Filename for evaluation map')